<a href="https://colab.research.google.com/github/bptripp/ai-course/blob/main/pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with a convolutional network
In this activity we will use a convolutional network for detection of pneumonia in chest radiographs. This is a more difficult task than classifying digits, so it will require a larger network. 

Training such a network from scratch would take several days, and it would require a large number of labelled examples. However, we can often get away with less training and less data if we begin with a network that has already been trained for a related task. 

For vision tasks, it is common to begin with a network that has been trained on ImageNet-1K, a dataset of over a million images of objects that are labelled with 1000 categories, including different kinds of animals, vehicles, etc. 

Detecting pneumonia in chest radiographs is quite different, even in terms of image statistics, so it is not clear in advance how well this approach will work for this use case, but we will see that it works reasonably well. 

You should run this code in Google Colab (or similar) on a GPU. Before proceeding, if you are using Colab, select "Change runtime type" from the "Runtime" menu, and select "GPU" from the "Hardware accelerator" drop-down list. Default values are fine for the other drop-downs. 

In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim


model = models.resnet50(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

We have now imported the necessary packages and downloaded a large convolutional network that has already been trained on ImageNet. Let's now move this network to the GPU that Colab has set aside for us. The code below should also print "cuda:0", which indicates that we are using a GPU. CUDA stands for Compute Unified Device Architecture. It is a low-level interface for running code on GPUs. PyTorch will use this interface to run the network on the GPU.  

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device)
print(device)

cuda:0


If you like, you can print the network structure by uncommenting and running the code below. 

In [ ]:
# print(model)

We will now make some changes to the network to prepare for training it on a new task. First, we will "freeze" the existing network parameters so that further training will not change them. We do this by telling PyTorch not to calculate the gradient of the loss with respect to these parameters during backpropagation. Second, we will replace the model's final "fully-connected" layer with two new layers. While the rest of the network is frozen, calculating features that were learned for ImageNet, these new layers will be trained to use these features for pneumonia inference.  

In [ ]:
# freeze existing network parameters
for parameter in model.parameters():
    parameter.requires_grad = False 

# replace final "fc" layer with two new layers that will be trained for pneumonia detection
model.fc = nn.Sequential(
               nn.Linear(2048, 64),
               nn.ReLU(inplace=True),
               nn.Linear(64, 2)).to(device)

Now we need labelled data for pneumonia detection. We will use data from this paper: 

Kermany, D. S., Goldbaum, M., Cai, W., Valentim, C. C., Liang, H., Baxter, S. L., ... & Zhang, K. (2018). Identifying medical diagnoses and treatable diseases by image-based deep learning. Cell, 172(5), 1122-1131.

There are two ways to proceed: 
1) The easiest way is to run the code below to download and unzip the dataset. 
2) Alternatively, you can obtain the data file from this link: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia Then, click on the file-folder icon to the left in Colab to upload the zip file to the Colab computer, and run only the unzip command below to unzip the file (comment out the first line). 

Option (1), which is more convenient, is available because the dataset owners have made the dataset available under a Creative Commons License. This copy of the dataset has not been altered. 

In [ ]:
!wget bptripp.com/Kermany-Chest-XRay-Data.zip
!unzip -q Kermany-Chest-XRay-Data.zip 

--2022-11-09 21:56:59--  http://bptripp.com/Kermany-Chest-XRay-Data.zip
Resolving bptripp.com (bptripp.com)... 64.90.50.171
Connecting to bptripp.com (bptripp.com)|64.90.50.171|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237670572 (1.2G) [application/zip]
Saving to: ‘Kermany-Chest-XRay-Data.zip’

Kermany-Chest-XRay- 100%[===================>]   1.15G  20.4MB/s    in 2m 11s  

2022-11-09 21:59:12 (9.04 MB/s) - ‘Kermany-Chest-XRay-Data.zip’ saved [1237670572/1237670572]



The next step is to create data loaders to load batches of data from these folders. 

In [ ]:
# Normalize images in the same way images are normalized for ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(10, shear=10), 
        transforms.ToTensor(),
        normalize
    ])

input_path = 'chest_xray/'
image_datasets = {
    'train': datasets.ImageFolder(input_path + 'train', transform),
    'validation': datasets.ImageFolder(input_path + 'test', transform)
}

dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=32, shuffle=True),  
    'validation': torch.utils.data.DataLoader(image_datasets['validation'], batch_size=32, shuffle=False)
}

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.005, weight_decay=1e-6)

In [ ]:
def train(model, train_loader, optimizer, epoch, device=None):
    model.train()
    total_loss = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if device is not None: 
          inputs = inputs.to(device)
          targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.CrossEntropyLoss()(outputs, targets)
        total_loss += loss
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print('Epoch: {} {}/{} Training loss: {:.6f}'.format(
                epoch,
                batch_idx * len(inputs),
                len(train_loader.dataset),
                loss))

    print('Training loss: {:.6f}'.format(total_loss / len(train_loader.dataset) * len(inputs)))


def test(model, test_loader, device=None):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
          if device is not None: 
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss += nn.CrossEntropyLoss()(outputs, targets)
            predictions = outputs.argmax(dim=1, keepdim=True)
            correct += predictions.eq(targets.view_as(predictions)).sum()

    loss = loss / len(test_loader.dataset) * len(inputs)

    print('Test loss: {:.6f}; Test accuracy: {}/{} ({:.1f}%)\n'.format(
        loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(5):
  train(model, dataloaders['train'], optimizer, epoch, device=device)
  test(model, dataloaders['validation'], device=device)

Epoch: 0 0/5232 Training loss: 0.743991
Epoch: 0 640/5232 Training loss: 0.364548
Epoch: 0 1280/5232 Training loss: 0.156028
Epoch: 0 1920/5232 Training loss: 0.174109
Epoch: 0 2560/5232 Training loss: 0.229006
Epoch: 0 3200/5232 Training loss: 0.074207
Epoch: 0 3840/5232 Training loss: 0.159176
Epoch: 0 4480/5232 Training loss: 0.352894
Epoch: 0 5120/5232 Training loss: 0.341475
Training loss: 0.131848
Test loss: 0.187664; Test accuracy: 535/624 (85.7%)

Epoch: 1 0/5232 Training loss: 0.065968
Epoch: 1 640/5232 Training loss: 0.721212
Epoch: 1 1280/5232 Training loss: 0.292268
Epoch: 1 1920/5232 Training loss: 0.457564
Epoch: 1 2560/5232 Training loss: 0.233795
Epoch: 1 3200/5232 Training loss: 0.035551
Epoch: 1 3840/5232 Training loss: 0.103989
Epoch: 1 4480/5232 Training loss: 0.356488
Epoch: 1 5120/5232 Training loss: 0.142546
Training loss: 0.098189
Test loss: 0.289512; Test accuracy: 498/624 (79.8%)

Epoch: 2 0/5232 Training loss: 0.216932
Epoch: 2 640/5232 Training loss: 0.13623